In [ ]:
import numpy as np
import pandas as pd
from plotly import graph_objects as go

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.nonparametric.smoothers_lowess import lowess

import skdh

In [ ]:
data = skdh.io.ReadCwa().predict(file="example_data.cwa")

In [ ]:
data

In [ ]:
accel_mag = np.linalg.norm(data['accel'], axis=1)


fig = go.Figure(data=go.Scatter(x=(pd.to_datetime(data['time']) - pd.to_datetime(data['time'])[0]), y=accel_mag))
fig.show()

In [ ]:
# load your data
time, accel = data['time'], data['accel']

gait = skdh.gait.GaitLumbar()
gait_res = gait.predict(time=time, accel=accel, height=1.8)

In [ ]:
# load your data
time, accel = data['time'], data['accel']

# setup a pipeline to first predict gait periods and then estimate gait
# metrics during those periods.
pipeline = skdh.Pipeline()
pipeline.add(skdh.context.PredictGaitLumbarLgbm())
pipeline.add(skdh.gait.GaitLumbar())

# get the results
res = pipeline.run(time=time, accel=accel, height=1.8)
# gait results are in res['GaitLumbar']

In [ ]:
res['GaitLumbar']

In [ ]:
pd.DataFrame(res['GaitLumbar']['gait speed'])

In [ ]:
gait_df = pd.DataFrame(res['GaitLumbar'])

In [ ]:
gait_df

In [ ]:
# gait_df.ffill(inplace=True)

In [ ]:
gait_df

In [ ]:
gait_df.columns


In [ ]:

# Convert 'IC Time' to datetime if not already
gait_df['IC Time'] = pd.to_datetime(gait_df['IC Time'])


# List of metrics you want to plot
metrics = [
    'stride time', 'stride time asymmetry', 'stance time',
       'stance time asymmetry', 'swing time', 'swing time asymmetry',
       'step time', 'step time asymmetry', 'initial double support',
       'initial double support asymmetry', 'terminal double support',
       'terminal double support asymmetry', 'double support',
       'double support asymmetry', 'single support',
       'single support asymmetry', 'm2 delta h', 'm2 delta h prime',
       'step length', 'step length asymmetry', 'stride length',
       'stride length asymmetry', 'gait speed', 'gait speed asymmetry',
       'cadence', 'm1 delta h', 'step length m1', 'step length m1 asymmetry',
       'stride length m1', 'stride length m1 asymmetry', 'gait speed m1',
       'gait speed m1 asymmetry', 'intra-step covariance - V',
       'intra-stride covariance - V', 'harmonic ratio - V', 'stride SPARC'
]

# Filter out rows with NaN values in the selected metrics
df_clean = gait_df[['IC Time'] + metrics].dropna().copy()


In [ ]:
# Setup big figure
rows, cols = 9, 4
fig, axes = plt.subplots(rows, cols, figsize=(24, 48), sharex=True)
axes = axes.flatten()

# Plot each metric
for i, metric in enumerate(metrics):
    ax = axes[i]
    sns.scatterplot(data=df_clean, x='IC Time', y=metric, ax=ax, s=10)
    ax.set_title(metric, fontsize=18)
    ax.tick_params(labelsize=12)

# Hide any extra axes
for j in range(len(metrics), len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:

# Group by 'Bout N' and compute the mean (or other stats) for each gait metric
#summary = gait_df.groupby('Bout N').mean(numeric_only=True)

# You can also get start times for each bout
#bout_starts = gait_df.groupby('Bout N')['Bout Starts'].first()

#summary['Bout Start Time'] = pd.to_datetime(bout_starts, unit='s')
#summary = summary.sort_values('Bout Start Time')

#print(summary[['Bout Start Time', 'gait speed', 'step length', 'cadence']].head())

In [ ]:
#import plotly.express as px
#px.line(summary, x="Bout Start Time", y="gait speed", title="Gait Speed over Time")
#px.line(summary, x="Bout Start Time", y="step length", title="Gait Speed over Time")
#px.line(summary, x="Bout Start Time", y="cadence", title="Gait Speed over Time")